## How Do You Work With APIs in SAS Viya?
This notebook supports the materials covered in the How Do You Work With APIs in SAS Viya? Ask the expert webinar.

### Use case
This collection leverages multiple API (both SAS Viya and CAS) to create an end-to-end example for modeling. It uploads data to CAS, imputes missing data, trains a decision tree model, creates a project for deployment, deploys the model to Micro Analytics Services (MAS) and scores new data.

## 1. Authenticate, variable assignment, and create CAS session

In [ ]:
import requests
import json
# The getpass module allows for a password prompt with variable assignment for the session
import getpass
#base_url = 'https://<URL>' -- fill in variable with your SAS Viya url
base_url = 'https://sasviyaserver.sas.com'
# Reference the Essential Preparation to configure secret(s) -- fill in variables from your SAS Viya environment
oAuthCliendId = 'client_id'
oAuthClientSecret = 'client_secret'

# Establish data & library variables - fill in variables from your SAS Viya environment -- uplodad the HMEQ dataset (google it) to the casuser caslib under the name TRAINING.sashdat
sourceCasLib = 'casuser'
sourceDataPathTable = 'TRAINING.sashdat'
destinationCasLib = 'casuser'
destinationDataPathTable = 'TRAINING_PYTHON'
imputeTable='TRAINING_IMPUTE_PYTHON'

print("If no errors, then variable assignment worked")


# prompt for credentials
user = input("Enter your username that you use to access SAS Viya:")
pw = getpass.getpass("Enter password - which will be reset to None:")
print('Requesting security token...')

# build OAuth API URL, then pass credentials to API to get OAuth Token
url = base_url + '/SASLogon/oauth/token'
headers = { 
    'content-type': 'application/x-www-form-urlencoded' 
    }
payload = 'grant_type=password&username=' + user + '&password=' + pw
response = requests.post(url, payload, headers=headers, auth=(oAuthCliendId, oAuthClientSecret), verify=False)
responseObj = json.loads(response.text)
oAuthAccessToken = responseObj['access_token']

# immediately reassign the pw variable to null since you are done with it
pw = None

# build Session API URL, then pass OAuth Token to API to get Session
url = base_url + '/cas-shared-default-http/cas/sessions'
headers = { 
    'Authorization': 'bearer ' + oAuthAccessToken 
}
response = requests.put(url, data=None, headers=headers, verify=False)
responseObj = json.loads(response.text)
casSessionId = responseObj['session']

# print the responses
print('\nThe token is:')
# I=if you'd like to hide your access token or session, please adjust the commented lines accordingly
# print('hidden')
print(oAuthAccessToken)
print('\nThe session is:')
# print('hidden')
print(casSessionId)
print('\nContinue to proceed if you recieved your OAuth Token and Session, otherwise you probably see a key error due to a bad password')

## 1.1 Create helper functions

In [ ]:
# Define a simple helper function to call API endpoint - not used in this example, but retained for code completeness
def callEndpoint():
    # pass headers to requests
    headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'application/json',
        'Accept':'application/json'
        }
    # call the API endpoint
    response = requests.post(url, headers=headers, json=payload, verify=False)
    # declare global responseObj
    global responseObj
    # recieve the responseObj
    responseObj = json.loads(response.text)

# Define a simple helper function to print the API payload
def printPayload():
    print('For debugging, the payload was defined as...')
    print(' -The API payload was:',payload)
    
# Define a simple helper function to print the API responses
def printResponse():
    status = responseObj.get('status')
    print('The response code is:')
    print(status)
    if status is None:
        print(' -The API response status was None')
    elif status == 0:
        print(' -The API response "status" is "0", indicating the action was successful.\n')
    else:
        print(' -The API response "status" is "non-zero",  indicating an error may have occurred')
        print(' -The API "log" is:',responseObj.get('log'))        
###
# need an else if the api log contains error like the below then indicate an error occurred
# -The API "log" is: ERROR: DATA step was not able to promote table water_cluster_ab in caslib CASUSER(anbout).  A promoted table with this name already exists.
###

# Define a simple helper function to print the API responseObj.log
def printResponseLog():
    print(' -The API "log" is:',responseObj.get('log'))
    
# Define a simple helper function to print the API response objects
def printResponseObj():
    print('\n -The full API response object is:',responseObj)
    
# Define a DataFrame table output
def printTable():
    tableColumns = pd.DataFrame(responseObj.get('results').get('ColumnInfo').get('rows'))
    print('\n -The table columns are:\n')
    display(tableColumns)

print("If no errors, then variable assignment worked")

## 2. Load data to CAS

In [ ]:
#PREREQUISITE - load HMEQ data set to casuser caslib; see step 1 for more details
# build full API URL -- perform the API action set -> /actions/table.loadTable
url = base_url + '/cas-shared-default-http/cas/sessions/' + casSessionId + '/actions/table.loadTable'

# pass payload to API
# path: specifies the file, directory, or table name.
payload = {
    'casLib': sourceCasLib,
    'path': sourceDataPathTable,
    'casout': {
        'caslib': destinationCasLib,
        'name': destinationDataPathTable,
        'promote': 'true'
        }
    }


headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'application/json',
        'Accept':'application/json'
        }
response = requests.post(url, headers=headers, json=payload, verify=False)

# declare global responseObj
global responseObj
# recieve the responseObj
responseObj = json.loads(response.text)
print(responseObj)

printResponse()
# Verify the payload
printPayload()

## 3. Impute missing data

In [ ]:
url = base_url + '/cas-shared-default-http/cas/sessions/' + casSessionId + '/actions/datapreprocess.impute'
payload = {
  "table": {
    "caslib": "casuser",
    "name": destinationDataPathTable
  },
  "vars": [
    "CLAGE", "CLNO", "DEBTINC", "DELINQ", "DEROG", "JOB", "LOAN", "MORTDUE", "NINQ", "REASON", "VALUE", "YOJ"],
  "outVarsNamePrefix": "",
  "methodContinuous": "median",
  "methodNominal": "mode",
  "casout": {
    "name": imputeTable,
    "caslib": "casuser",
    "promote": True
  },
  "copyvars": [
    "BAD"
  ]
}
headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'application/json',
        'Accept':'application/json'
        }
response = requests.post(url, headers=headers, json=payload, verify=False)

# declare global responseObj
global responseObj
# recieve the responseObj
responseObj = json.loads(response.text)
print(responseObj)

printResponse()
# Verify the payload
printPayload()

## 4. Train decision tree model

In [ ]:
url = base_url + '/cas-shared-default-http/cas/sessions/' + casSessionId + '/actions/decisionTree.dtreeTrain'

payload = {
  "inputs": [
    "CLAGE",
    "CLNO",
    "DEBTINC"
  ],
  "nominals": [
    "BAD"
  ],
  "table": {
    "name": imputeTable,
    "caslib": "casuser"
  },
  "target": "BAD",
  "casout": {
    "name": "dt_trained_python",
    "caslib": "casuser",
    "replace": True
  },
  "code": {
    "casout": {
      "caslib": "casuser",
      "name": "dt_code_python",
      "promote": True
    }
  }
}
headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'application/json',
        'Accept':'application/json'
        }
response = requests.post(url, headers=headers, json=payload, verify=False)

# declare global responseObj
global responseObj
# recieve the responseObj
responseObj = json.loads(response.text)
print(responseObj)

printResponse()
# Verify the payload
printPayload()

## 5. Save code from decision tree model

In [ ]:
url = base_url + '/cas-shared-default-http/cas/sessions/' + casSessionId + '/actions/decisionTree.dtreeCode'

payload = {
  "modeltable": {
    "caslib": "casuser",
    "name": "dt_trained_python"
  }
}
headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'application/json',
        'Accept':'application/json'
        }
response = requests.post(url, headers=headers, json=payload, verify=False)

# declare global responseObj
global responseObj
# recieve the responseObj
responseObj = json.loads(response.text)
print(responseObj)

printResponse()
# Verify the payload
printPayload()


## 6. Fetch score code

In [ ]:
url = base_url + '/cas-shared-default-http/cas/sessions/' + casSessionId + '/actions/table.fetch'

payload = {
  "table": {
    "caslib": "casuser",
    "name": "dt_code_python"
  },
  "fetchVars": [
    "DataStepSrc"
  ]
}
headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'application/json',
        'Accept':'application/json'
        }
response = requests.post(url, headers=headers, json=payload, verify=False)
sas_code = response.json()['results']['Fetch']['rows'][0][1]


# declare global responseObj
global responseObj
# recieve the responseObj
responseObj = json.loads(response.text)
print(responseObj)

printResponse()
# Verify the payload
printPayload()

## 7. Get repository information

In [ ]:
url = base_url + '/modelRepository/repositories?name=Public'

payload={}

headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        }
response = requests.get(url, headers=headers, json=payload, verify=False).json()
m_repo_id = response["items"][0]["id"]
repo_fid = response["items"][0]["folderId"]
print(m_repo_id)
print(repo_fid)

print(response)

printResponse()
# Verify the payload
printPayload()

## 8. Create SAS code file

In [ ]:
url = base_url + '/files/files?parentFolderUri=/folders/folders/' + repo_fid + '&filename=sas_code.sas&Content-Disposition=attachment'
 

headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'text/plain',
        'Accept': 'application/vnd.sas.file+json'
        }
response = requests.post(url, headers=headers, json=sas_code, verify=False)

# declare global responseObj
global responseObj
# recieve the responseObj
responseObj = json.loads(response.text)
print(responseObj)

printResponse()
# Verify the payload
printPayload()

## 9. Create publishing destination

In [ ]:
url = base_url + '/modelPublish/destinations'


payload = {
    "name" : "realtime_deployment_python",
    "destinationType" : "microAnalyticService"
}

headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'application/vnd.sas.models.publishing.destination.mas+json',
        'Accept': 'application/vnd.sas.models.publishing.destination.mas+json',
        }
response = requests.post(url, headers=headers, json=payload, verify=False)


# declare global responseObj
global responseObj
# recieve the responseObj
responseObj = json.loads(response.text)
print(response)

printResponse()
# Verify the payload
printPayload()

## 10. Create model project

In [ ]:
url = base_url + '/modelRepository/projects'

payload = {
  "name": "e-to-e-model-project-python",
  "repositoryId": m_repo_id,
  "folderId": repo_fid
}

headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'application/vnd.sas.models.project+json',
        'Accept':'application/json'
        }
response = requests.post(url, headers=headers, json=payload, verify=False).json()
project_id=response["id"]
print(project_id)

print(response)

printResponse()
# Verify the payload
printPayload()

## 11. Create model

In [ ]:
url = base_url + '/modelRepository/models'

payload = {
  "name": "model created by api in python",
  "scoreCodeType": "dataStep",
  "eventProbVar": "DT_BAD_PredP",
  "targetLevel": "binary",
  "targetVariable": "DT_BAD",
  "projectId": project_id,
  "folderId": repo_fid,
  "repositoryId": m_repo_id,
  "function": "classification",
  "inputVariables": [
    {
      "name": "CLAGE",
      "type": "integer"
    },
    {
      "name": "CLNO",
      "type": "integer"
    },
    {
      "name": "DEBTINC",
      "type": "integer"
    }
  ],
  "outputVariables": [
    {
      "name": "DT_BAD",
      "type": "integer"
    },
    {
      "name": "DT_BAD_PredP",
      "type": "decimal"
    }
  ]
}


headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'application/vnd.sas.models.model+json',
        'Accept':'application/json'
        }
response = requests.post(url, headers=headers, json=payload, verify=False).json()
model_id=response["items"][0]["id"]
print(model_id)

print(response)

printResponse()
# Verify the payload
printPayload()

## 11.1 Get model info

In [ ]:
url = base_url + '/modelRepository/models/' + model_id

payload={}
headers = {
  'Authorization': 'bearer ' + oAuthAccessToken
}

response = requests.get(url, headers=headers, data=payload, verify=False)

# declare global responseObj
global responseObj
# recieve the responseObj
responseObj = json.loads(response.text)
print(responseObj)

printResponse()
# Verify the payload
printPayload()

## 12. Add model files

In [ ]:
url = base_url + '/modelRepository/models/' + model_id + '/contents?name=score.sas'

payload=sas_code

headers = {
  'Authorization': 'bearer ' + oAuthAccessToken
}

response = requests.post(url, headers=headers, data=payload, verify=False)

# declare global responseObj
global responseObj
# recieve the responseObj
responseObj = json.loads(response.text)
print(responseObj)

printResponse()
# Verify the payload
printPayload()

## 13. Publish model

In [ ]:
url = base_url + '/modelManagement/publish'

payload = {
  "name": "APIModel-Python",
  "notes": "Publish models using Python",
  "modelContents": [
    {
      "modelName": "model_api_python",
      "sourceUri": "/modelRepository/models/" + model_id,
      "publishLevel": "model"
    }
  ],
  "destinationName": "realtime_deployment_python"
}

headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'application/vnd.sas.models.publishing.request.asynchronous+json'
        }
response = requests.post(url, headers=headers, json=payload, verify=False).json()
m_pub_id=response["id"]
m_publish_name=response["name"]
print(m_pub_id)
print(m_publish_name)

print(response)

printResponse()
# Verify the payload
printPayload()

## 13.1. Check published model

In [ ]:
url = base_url + '/modelPublish/models/' + m_pub_id

payload={}
headers = {
  'Authorization': 'bearer ' + oAuthAccessToken
}

response = requests.get(url, headers=headers, data=payload, verify=False)

# declare global responseObj
global responseObj
# recieve the responseObj
responseObj = json.loads(response.text)
print(responseObj)

printResponse()
# Verify the payload
printPayload()

## 14. Score new data

In [ ]:
url = base_url + '/microanalyticScore/modules/' + m_publish_name + '/steps/score'

payload = {
  "inputs": [
    {
      "name": "clage",
      "value": 20
    },
    {
      "name": "clno",
      "value": 100
    },
    {
      "name": "debtinc",
      "value": 2
    }
  ]
}
headers = {
        'Authorization': 'bearer ' + oAuthAccessToken,
        'Content-Type': 'application/json',
        }
response = requests.post(url, headers=headers, json=payload, verify=False)

# declare global responseObj
global responseObj
# recieve the responseObj
responseObj = json.loads(response.text)
print(responseObj)

printResponse()
# Verify the payload
printPayload()

## 15. End CAS session

In [ ]:
url = base_url + '/casManagement/servers/cas-shared-default/sessions/' + casSessionId

payload={}
headers = {
        'Authorization': 'bearer ' + oAuthAccessToken
        }

response = requests.delete(url, headers=headers, json=payload, verify=False)

print(response.text)